In [1]:
!pip install -U pip
!pip install accelerate==0.20.3
!pip install appdirs==1.4.4
!pip install bitsandbytes==0.39.0
!pip install datasets==2.10.1
!pip install fire==0.5.0
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install torch==2.0.0
!pip install sentencepiece==0.1.97
!pip install tensorboardX==2.6
!pip install gradio==3.23.0

In [2]:
import os
import sys

# import fire
# import gradio as gr
import torch
import transformers
from peft import PeftModel
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

from utils.callbacks import Iteratorize, Stream
from utils.prompter import Prompter


device_no = 0

if torch.cuda.is_available():
    device = f"cuda:{device_no}"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:  # noqa: E722
    pass


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!mkdir adapter_model

In [5]:
!cp -r /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b/adapter_model/ adapter_model

In [ ]:
# template folder
# alpaca.json
# utils folder

In [3]:
prompt_template = "alpaca"
base_model = "decapoda-research/llama-7b-hf"
load_8bit = False
lora_weights = "adapter_model"
prompter = Prompter(prompt_template)
tokenizer = LlamaTokenizer.from_pretrained(base_model)
# if device == "cuda:13":
add_eos_token: bool = False
cutoff_len = CUTOFF_LEN = 256
train_on_inputs = False
prompt_template_name: str = "alpaca"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    # quantization_config=bnb_config,
    device_map={"": device_no},
)
# model = LlamaForCausalLM.from_pretrained(
#     base_model,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )
model = PeftModel.from_pretrained(
    model,
    lora_weights,
    torch_dtype=torch.float16,
)
print("reached here")


# unwind broken decapoda-research config
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

model.eval()
model = model.merge_and_unload()
model = torch.compile(model)
print("reached here")
print(model.training)
add_eos_token: bool = False
cutoff_len = CUTOFF_LEN = 256
train_on_inputs = False
prompt_template_name: str = "alpaca"
model.eval()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

reached here
reached here
True


In [6]:
model.training

False

In [38]:
def evaluate(
    input_ids,
    # instruction,
    # input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    # max_new_tokens is set to 8, as for our model, only a single word prediction is needed
    max_new_tokens=32,
    stream_output=False,
    **kwargs,
):
    # prompt = prompter.generate_prompt(instruction, input)
    # inputs = tokenizer(prompt, return_tensors="pt")
    # input_ids = inputs["input_ids"].to(device)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )

    generate_params = {
        "input_ids": input_ids,
        "generation_config": generation_config,
        "return_dict_in_generate": True,
        "output_scores": True,
        "max_new_tokens": max_new_tokens,
    }

    if stream_output:
        # Stream the reply 1 token at a time.
        # This is based on the trick of using 'stopping_criteria' to create an iterator,
        # from https://github.com/oobabooga/text-generation-webui/blob/ad37f396fc8bcbab90e11ecf17c56c97bfbd4a9c/modules/text_generation.py#L216-L243.

        def generate_with_callback(callback=None, **kwargs):
            kwargs.setdefault(
                "stopping_criteria", transformers.StoppingCriteriaList()
            )
            kwargs["stopping_criteria"].append(
                Stream(callback_func=callback)
            )
            with torch.no_grad():
                model.generate(**kwargs)

        def generate_with_streaming(**kwargs):
            return Iteratorize(
                generate_with_callback, kwargs, callback=None
            )

        with generate_with_streaming(**generate_params) as generator:
            for output in generator:
                # new_tokens = len(output) - len(input_ids[0])
                decoded_output = tokenizer.decode(output)

                if output[-1] in [tokenizer.eos_token_id]:
                    break

                return prompter.get_response(decoded_output)
        return  # early return for stream_output

    # Without streaming
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return prompter.get_response(output)



def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    # result["labels"] = result["input_ids"].copy()

    return result


def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        # data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    # if not train_on_inputs:
    #     user_prompt = prompter.generate_prompt(
    #         data_point["instruction"], data_point["input"]
    #     )
    #     tokenized_user_prompt = tokenize(
    #         user_prompt, add_eos_token=add_eos_token
    #     )
    #     user_prompt_len = len(tokenized_user_prompt["input_ids"])

    #     if add_eos_token:
    #         user_prompt_len -= 1

    #     tokenized_full_prompt["labels"] = [
    #         -100
    #     ] * user_prompt_len + tokenized_full_prompt["labels"][
    #         user_prompt_len:
    #     ]  # could be sped up, probably
    return tokenized_full_prompt


In [46]:
from tqdm import tqdm
dataset_name = "reddit"
data_path = f"{dataset_name}.json"
from datasets import load_dataset
data = load_dataset("json", data_files=data_path)


# train_val = data["train"].train_test_split(
#     test_size=200, shuffle=True, seed=42
# )
data = (
    data["train"].shuffle().map(generate_and_tokenize_prompt)
)
# data.set_format("torch", columns=["input", "input_ids", "attention_mask", "output"])
# val_data = (
#     train_val["test"].shuffle().map(generate_and_tokenize_prompt)
# )
print(data[0])
# answer = evaluate(data[0]["instruction"], data[0]["input"]

  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3337 [00:00<?, ? examples/s]

{'instruction': 'Detect if the text is hateful or neutral.', 'input': '@USER @USER “infestation” last week and today it’s “healing”.  How does that work?', 'output': 'NEUTRAL', 'input_ids': [0, 13866, 338, 385, 15278, 393, 16612, 263, 3414, 29892, 3300, 2859, 411, 385, 1881, 393, 8128, 4340, 3030, 29889, 14350, 263, 2933, 393, 7128, 2486, 1614, 2167, 278, 2009, 29889, 13, 13, 2277, 29937, 2799, 4080, 29901, 13, 6362, 522, 565, 278, 1426, 338, 298, 22279, 470, 21104, 29889, 13, 13, 2277, 29937, 10567, 29901, 13, 29992, 11889, 732, 11889, 1346, 7192, 342, 362, 30024, 1833, 4723, 322, 9826, 372, 30010, 29879, 1346, 354, 12818, 8643, 29871, 1128, 947, 393, 664, 29973, 13, 13, 2277, 29937, 13291, 29901, 13, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [47]:
inputs = []
predictions = []
real_answers = []

In [48]:
inputs

[]

In [49]:
for i in tqdm(range(100)):
    # print(real_answer)
    try:
      model_answer = evaluate(data[i]["input_ids"])
      real_answer = data[i]["output"]
      input_text = data["input"][i]
    except:
      continue
    inputs.append(input_text)
    real_answers.append(real_answer)
    predictions.append(model_answer)


# from pprint import pprint
# pprint(predictions)
# print("="*100)
# pprint(real_answers)
# print("="*100)
# predictions = [evaluate(data[i]["instruction"], data[i]["input"]) for i in range(100)]

100%|██████████| 100/100 [03:37<00:00,  2.18s/it]


In [ ]:
# current_test= {"dataset": dataset_name, "inputs": inputs, "predictions": predictions, "real_answers": real_answers}

In [52]:
import pickle

# a = {'hello': 'world'}
pickle_file_name = f'llama_1epoch_{dataset_name}_100_test.pickle'
with open(pickle_file_name, 'wb') as handle:
    pickle.dump(current_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
!cp .pickle /content/drive/MyDrive/ml-models/llama_training_reddit_colab_1_ep_128b/

In [14]:
real_answers

['NEUTRAL', 'HATEFUL', 'NEUTRAL']

In [10]:
real_answers

['NEUTRAL', 'HATEFUL', 'NEUTRAL']

In [15]:
predictions

['<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk>NEUT',
 '<unk> ATEHATEF<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk>NEUT',
 '<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk> NEUTRAL<unk>NEUT']

In [11]:
predictions

['<unk> NEUTRAL<unk>NEUTR',
 '<unk> ATEHATEF<unk>NEUTR',
 '<unk> NEUTRAL<unk>NEUTR']